In [1]:
%load_ext rpy2.ipython



/Users/coltongarelli/anaconda3/envs/GeneNetworking/lib/python3.7/site-packages/rpy2/ipython/rmagic.py:85: UserWarning: The Python package `pandas` is strongly recommended when using `rpy2.ipython`. Unfortunately it could not be loaded, as we did not manage to load `numpy` in the first place (error: No module named 'numpy').
  'in the first place (error: %s).' % str(ie))


In [2]:
%%R

library(org.Hs.eg.db)

R[write to console]: Loading required package: AnnotationDbi

R[write to console]: Loading required package: stats4

R[write to console]: Loading required package: BiocGenerics

R[write to console]: Loading required package: parallel

R[write to console]: 
Attaching package: ‘BiocGenerics’


R[write to console]: The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB


R[write to console]: The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


R[write to console]: The following objects are masked from ‘package:base’:

    Filter, Find, Map, Position, Reduce, anyDuplicated, append,
    as.data.frame, basename, cbind, colnames, dirname, do.call,
    duplicated, eval, evalq, get, grep, grepl, intersect, is.unsorted,
    lapply, mapply, match, mget, order, paste, pmax, pmax.int, pmin

In [3]:
%%R
e_ctcl <- read.csv('/Users/coltongarelli/Desktop/GSE113113_stage1_DE.csv')
filt <- c(
    'gene', 'logFC', 'adj.P.Val')
l_ctcl <- read.csv('/Users/coltongarelli/Desktop/GSE113113_stage2_DE.csv')
dle <- read.csv('/Users/coltongarelli/Desktop/GSE95474 DLE diffex collapsed probes.csv')
e_ctcl <- e_ctcl[ ,filt]
l_ctcl <- l_ctcl[ ,filt]
dle <- dle[ ,filt]
dle <- dle[abs(dle$logFC) > 1.0,]
dle <- dle[dle$adj.P.Val < 0.05,]

e_ctcl <- e_ctcl[abs(e_ctcl$logFC) > 1.0,]
e_ctcl <- e_ctcl[e_ctcl$adj.P.Val < 0.05,]

l_ctcl <- l_ctcl[abs(l_ctcl$logFC) > 1.0,]
l_ctcl <- l_ctcl[l_ctcl$adj.P.Val < 0.05,]

In [4]:
%%R
dle_simp <- dle[c('gene', 'logFC')]
ec_simp <- e_ctcl[c('gene', 'logFC')]
lc_simp <- l_ctcl[c('gene', 'logFC')]

In [5]:
%%R


rownames(dle_simp) <- dle_simp$gene
# dle_simp$gene <- NULL
dle_simp <- dle_simp[order(dle_simp$logFC, decreasing=TRUE),]
# dle_simp

rownames(ec_simp) <- ec_simp$gene
# ec_simp$gene <- NULL
ec_simp <- ec_simp[order(ec_simp$logFC, decreasing=TRUE),]

rownames(lc_simp) <- lc_simp$gene
# lc_simp$gene <- NULL
lc_simp <- lc_simp[order(lc_simp$logFC, decreasing=TRUE),]

In [6]:
%%R
# IDs
dle_symbols <- clusterProfiler::bitr(rownames(dle_simp), fromType='ALIAS', toType='ENTREZID', OrgDb='org.Hs.eg.db')
dle_symbols$gene <- dle_symbols$ALIAS
dle_symbols$ALIAS <- NULL
dle_entrez <- merge(dle_simp, dle_symbols, by.y='gene')

R[write to console]: 

R[write to console]: Registered S3 method overwritten by 'enrichplot':
  method               from
  fortify.enrichResult DOSE

R[write to console]: 'select()' returned 1:many mapping between keys and columns



In [7]:
%%R
options(max.print=1000)


dle_agg <- aggregate(dle_simp$logFC,
          by = list(gene = dle_simp$gene),
          FUN = mean,
          na.rm = TRUE)
# dle_agg <- dle_agg[order(dle_agg$x, decreasing=TRUE),]
# dle_agg
# dle_entrez
# rownames(dle_agg) <- dle_agg$entrez
# dle_agg$entrez <- NULL
rownames(dle_simp)<-dle_simp$gene
dle_simp$gene <- NULL
dle_vec <- unlist(dle_simp)
names(dle_vec) <- rownames(dle_simp)
# dle_agg
dle_gsego <- clusterProfiler::gseGO(geneList= dle_vec,
              OrgDb        = 'org.Hs.eg.db',
              ont          = "BP",
              nPerm        = 1000,
              minGSSize    = 100,
              maxGSSize    = 500,
              pvalueCutoff = 0.05,
              verbose      = FALSE,
              keyType      = 'SYMBOL')

In [ ]:
%%R


In [ ]:
%%R

dle_gsego
dle_vec 

# pdf(file='/Users/coltongarelli/Desktop/dle_go_graph_GSE95474.pdf', height=15, width=15)
# dle_gog <- clusterProfiler::plotGOgraph(dle_gsego[100:nrow(dle_gsego),], firstSigNodes=100)
# print(dle_gog)
# dev.off()

In [ ]:
%%R
pdf(file='/Users/coltongarelli/Desktop/dle_circ_cnet_fc_GSE95474.pdf', height=15, width=15)
dle_gog <- clusterProfiler::cnetplot(y, foldChange=dle_vec, showCategory=10, colorEdge=TRUE)
print(dle_gog)
dev.off()

In [8]:
%%R
# install.packages("devtools")
# devtools::install_github("YuLab-SMU/clusterProfiler.dplyr", repos='http://cran.us.r-project.org')
library(clusterProfiler.dplyr)
y <- clusterProfiler.dplyr::arrange(dle_gsego, abs(NES)) %>% 
        clusterProfiler.dplyr::group_by(sign(NES)) %>% 
        clusterProfiler.dplyr::slice(1:200)
DOSE::upsetplot(y)
# clusterProfiler::cnetplot(y)

R[write to console]: 
Attaching package: ‘clusterProfiler.dplyr’


R[write to console]: The following object is masked from ‘package:AnnotationDbi’:

    select


R[write to console]: The following object is masked from ‘package:IRanges’:

    slice


R[write to console]: The following object is masked from ‘package:S4Vectors’:

    rename


R[write to console]: The following object is masked from ‘package:stats’:

    filter


R[write to console]: Error: 'upsetplot' is not an exported object from 'namespace:DOSE'

R[write to console]: In addition: 
R[write to console]: Warning messages:

R[write to console]: 1: 
R[write to console]: In clusterProfiler::bitr(rownames(dle_simp), fromType = "ALIAS",  :
R[write to console]: 
 
R[write to console]:  20.59% of input gene IDs are fail to map...

R[write to console]: 2: 
R[write to console]: In fgsea(pathways = geneSets, stats = geneList, nperm = nPerm, minSize = minGSSize,  :
R[write to console]: 
 
R[write to console]:  There were 1 pathway


Error: 'upsetplot' is not an exported object from 'namespace:DOSE'


In [ ]:
%%R

dle_kegg <- clusterProfiler::compareCluster(rownames(dle_simp), fun="enrichKEGG", organism='hsa', pvalueCutoff=0.05)
clusterProfiler::emapplot(dle_kegg)

In [ ]:
%%R
dle_symbols <- clusterProfiler::bitr(rownames(dle_simp), fromType='ALIAS', toType='KEGGID', OrgDb='org.Hs.eg.db')

# clusterProfiler::emapplot(dle_kegg)


In [ ]:
%%R
library(DOSE)
pdf(file='/Users/coltongarelli/Desktop/dle_emap_fc_GSE95474.pdf', height=15, width=15)
dle_gog <- clusterProfiler::emapplot(dle_gsego[100:,])
print(dle_gog)
dev.off()

In [ ]:
%%R
ec_gsego <- clusterProfiler::gseGO(geneList= ec_simp,
              OrgDb        = 'org.Hs.eg.db',
              ont          = "BP",
              nPerm        = 1000,
              minGSSize    = 100,
              maxGSSize    = 500,
              pvalueCutoff = 0.05,
              verbose      = FALSE)

In [ ]:
%%R
lc_gsego <- clusterProfiler::gseGO(geneList= lc_simp,
              OrgDb        = 'org.Hs.eg.db',
              ont          = "BP",
              nPerm        = 1000,
              minGSSize    = 100,
              maxGSSize    = 500,
              pvalueCutoff = 0.05,
              verbose      = FALSE)

In [ ]:
%%R



In [ ]:
%%R

